# Thompson sampling

From Wikipedia [^1]
> Thompson sampling,[1][2][3] named after William R. Thompson, is a heuristic for choosing actions that addresses the exploration-exploitation dilemma in the multi-armed bandit problem. It consists of choosing the action that maximizes the expected reward with respect to a randomly drawn belief.

[^1]: https://en.wikipedia.org/wiki/Thompson_sampling

# Resources
- https://gdmarmerola.github.io/ts-for-bernoulli-bandit/
- https://github.com/gdmarmerola/interactive-intro-rl/blob/master/notebooks/ts_for_multi_armed_bandit.ipynb

In [1]:
import numpy as np
import matplotlib.pyplot as plt


class MAB:
    def __init__(self, bandit_probs):
        self.bandit_probs = bandit_probs

    def draw(self, k):
        reward = np.random.binomial(1, self.bandit_probs[k]) # Returns either 0 or 1
        regret = np.max(self.bandit_probs) - self.bandit_probs[k]
        return reward, regret

In [2]:
class eGreedyPolicy:
    def __init__(self, epsilon):
        self.epsilon = epsilon

    def choose_bandit(self, k_array, reward_array, n_bandits):
        total_success = reward_array.sum(axis=1)
        total_count = k_array.sum(axis=1)
        success_ratio = total_success/total_count
        
        best_action = np.argmax(success_ratio)
        if np.random.random() < self.epsilon:
            # Returning random action, excluding best
            return np.random.choice(list(range(n_bandits)) - best_action)
        else:
            # Returning best greedy action.
            return best_action

In [3]:
# e-greedy policy
class UCBPolicy:
    
    # initializing
    def __init__(self):
        
        # nothing to do here
        pass
    
    # choice of bandit
    def choose_bandit(self, k_array, reward_array, n_bandits):
        
        # sucesses and total draws
        success_count = reward_array.sum(axis=1)
        total_count = k_array.sum(axis=1)
        
        # ratio of sucesses vs total
        success_ratio = success_count/total_count
        
        # computing square root term
        sqrt_term = np.sqrt(2*np.log(np.sum(total_count))/total_count)
        
        # returning best greedy action
        return np.argmax(success_ratio + sqrt_term)    

In [4]:
class TSPolicy:
    
    # initializing
    def __init__(self):
        
        # nothing to do here
        pass
    
    # choice of bandit
    def choose_bandit(self, k_array, reward_array, n_bandits):
        # list of samples, for each bandit
        samples_list = []
        
        # sucesses and failures
        success_count = reward_array.sum(axis=1)
        failure_count = k_array.sum(axis=1) - success_count
                    
        # drawing a sample from each bandit distribution
        samples_list = [np.random.beta(1 + success_count[bandit_id], 1 + failure_count[bandit_id]) for bandit_id in range(n_bandits)]
                                
        # returning bandit with best sample
        return np.argmax(samples_list)    

In [9]:
import numpy as np

# defining a set of bandits with known probabilites
bandit_probs = [0.35, 0.40, 0.30, 0.25]

# instance of our MAB class
mab = MAB(bandit_probs)

# policy
egreedy_policy = eGreedyPolicy(0.1)
ucb_policy = UCBPolicy()
ts_policy = TSPolicy()

In [10]:
def random_policy(k_array, reward_array, N_BANDITS):
    return np.random.choice(range(N_BANDITS))

In [15]:
N_DRAWS = 5000
N_BANDITS = len(mab.bandit_probs)

policies = [random_policy, egreedy_policy.choose_bandit, ucb_policy.choose_bandit, ts_policy.choose_bandit]

for policy in policies:
    k_array = np.zeros((N_BANDITS, N_DRAWS))
    reward_array = np.zeros((N_BANDITS, N_DRAWS))
    total_regret = 0
    
    for i in range(N_DRAWS):
        k = policy(k_array, reward_array, N_BANDITS)
        reward, regret = mab.draw(k)
        k_array[k, i] = 1
        reward_array[k, i] = reward
        total_regret += regret
    print(k_array.sum(axis=1), reward_array.sum(axis=1), total_regret)

[1297. 1264. 1199. 1240.] [472. 511. 352. 305.] 370.7500000000091


/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_14289/1459101977.py:8: RuntimeWarning: invalid value encountered in divide
  success_ratio = total_success/total_count


[1558. 3016.  223.  203.] [ 529. 1241.   58.   49.] 130.64999999999668


/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_14289/2987556313.py:18: RuntimeWarning: invalid value encountered in divide
  success_ratio = success_count/total_count
/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_14289/2987556313.py:21: RuntimeWarning: divide by zero encountered in log
  sqrt_term = np.sqrt(2*np.log(np.sum(total_count))/total_count)
/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_14289/2987556313.py:21: RuntimeWarning: invalid value encountered in sqrt
  sqrt_term = np.sqrt(2*np.log(np.sum(total_count))/total_count)
/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_14289/2987556313.py:21: RuntimeWarning: invalid value encountered in divide
  sqrt_term = np.sqrt(2*np.log(np.sum(total_count))/total_count)
/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_14289/2987556313.py:21: RuntimeWarning: divide by zero encountered in divide
  sqrt_term = np.sqrt(2*np.log(np.sum(total_count))/total_count)


[ 968. 3061.  566.  405.] [ 329. 1219.  169.  108.] 165.75000000000097
[ 522. 4192.  249.   37.] [ 181. 1668.   83.    6.] 56.549999999999635
